In [ ]:
"""
    This cell imports are the packages used.
"""
import pandas as pd
import numpy as np
import time
import math
from datetime import datetime,timedelta
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from keras import backend as kb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import initializers
from sklearn.metrics import mean_squared_error

In [ ]:
class BNNOptionPricing():
    """
        The BNNOptionPricing class uses the Bayesian Neural Network (BNN) to price options.
    """


    def __init__(self,
                 data,
                 feature_names,
                 target,
                 alpha = 10**(-3),
                 beta = 1,
                 hidden_nodes = [32, 16],
                 learning_rate = 0.001,
                 batch_size = 512):
        """
            __init__ defines all the variables and functions used.
            E.g. alpha and beta are the regularization parameters.
        """
      
        self.data = data
        self.hidden_nodes = hidden_nodes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.alpha = alpha
        self.beta = beta
        
        #Checking that the inputs are correctly given
        if feature_names != None and target != None:
            self.feature_names = feature_names
            self.target = target
        else:
            raise ValueError('Please provide the features and the target')

        #Create the inputs for the Neural Network
        self.model_inputs = self.create_model_inputs(self.feature_names)
        
        #Initialze the Bayesian Neural Network
        self.BNN = BNN(self.hidden_nodes, self.learning_rate, self.batch_size, self.alpha)
        self.model = self.BNN.create_bnn_model(len(self.data), self.model_inputs)


    def df_to_dataset(self, data = None, shuffle = True):
        '''
            This function converts the given data (Pandas Dataframe) into a 
            tensorflow data which is needed as a input for the Neural Network.
        '''
        temp = self.feature_names + [self.target]
        dataframe = data[temp].copy()
        labels = dataframe.pop(self.target)
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(self.batch_size)
        return ds

    def create_model_inputs(self, features):
        """
            This function returns the model inputs (in our case: Moneyness and Maturity)
            for the Neural Network.
        """
        inputs = {}
        for feature_name in features:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(1,), dtype=tf.float32
            )
        return inputs

    def custom_loss_function(self, y_true, y_pred):
        """
            The custom_loss_function returns the loss value for our specified loss function. 
            self.beta is used as a regularization parameter.
            The loss function can be found in the Bayesian Neural Network section in our report.
        """
        return self.beta*kb.sum(kb.square(y_true - y_pred))

    def compile_bnn(self):
        """
            Here the Neural Network is compiled with the optimizer, loss function 
            and we also define RMSE as metric.
        """
        self.model.compile(
            optimizer=keras.optimizers.RMSprop(learning_rate = self.BNN.learning_rate),
            loss = self.custom_loss_function,
            metrics=[keras.metrics.RootMeanSquaredError()],
        )

    def train_and_evaluate_bnn(self, num_epochs, train_data = None, val_data = None):
        """
            This function trains the model on th training data and validation data 
            with the specified numbers of epochs.
        """
        self.compile_bnn()
        print("Start training the model...")
        self.model.fit(train_data, epochs=num_epochs, validation_data=val_data, verbose = 2)
        print("Model training finished.")

    def create_est_train_test(self, period):
        """
            This function splits the data in training, validation and test data for the estimation errors.
        """
        if period == 'pre-crisis':
            train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2006,7,1))]
            val = self.data[(self.data['date'] >= pd.Timestamp(2006,7,1)) & (self.data['date'] < pd.Timestamp(2007,1,1))]
            test = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2007,1,1))]
        elif period == 'crisis':
            train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2009,7,1))]
            val = self.data[(self.data['date'] >= pd.Timestamp(2009,7,1)) & (self.data['date'] < pd.Timestamp(2010,1,1))]
            test = self.data[(self.data['date'] >= pd.Timestamp(2007,1,1)) & (self.data['date'] < pd.Timestamp(2010,1,1))]
        elif period == 'post-crisis':
            train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2012,7,1))]
            val = self.data[(self.data['date'] >= pd.Timestamp(2012,7,1)) & (self.data['date'] < pd.Timestamp(2013,1,1))]
            test = self.data[(self.data['date'] >= pd.Timestamp(2010,1,1)) & (self.data['date'] < pd.Timestamp(2013,1,1))]
        else:
            raise ValueError('Not a valid string for period')
        return train, val, test


    def create_pred_train_test(self, period):
        """
            This function splits the data in training, validation and test data for the prediction errors.
            The prediction period is about half a year.
        """
        if period == 'pre-crisis':
          train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2006,1,1))]
          val = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2006,7,1))]
          test = self.data[(self.data['date'] >= pd.Timestamp(2006,7,1)) & (self.data['date'] < pd.Timestamp(2007,1,1))]
        elif period == 'crisis':
          train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2009,1,1))]
          val = self.data[(self.data['date'] >= pd.Timestamp(2009,1,1)) & (self.data['date'] < pd.Timestamp(2009,7,1))]
          test = self.data[(self.data['date'] >= pd.Timestamp(2009,7,1)) & (self.data['date'] < pd.Timestamp(2010,1,1))]
        elif period == 'post-crisis':
          train = self.data[(self.data['date'] >= pd.Timestamp(2003,1,1)) & (self.data['date'] < pd.Timestamp(2012,1,1))]
          val = self.data[(self.data['date'] >= pd.Timestamp(2012,1,1)) & (self.data['date'] < pd.Timestamp(2012,7,1))]
          test = self.data[(self.data['date'] >= pd.Timestamp(2012,7,1)) & (self.data['date'] < pd.Timestamp(2013,1,1))]
        else:
          raise ValueError('Not a valid string for period')
        return train, val, test
                

    def cal_prediction_errors(self, epochs, period = None, iterations = 50):
        '''
            The cal_prediction_error function is actually one of the main functions which combines a lot of 
            other (pre-)defined functions to calculate the prediction error. The Bayesian Neural Network is
            trained in this function and thereafter, the prices are predicted and the errors are calculated.
        '''
        print('Model cannot be trained beforehand - delete trained model before calling this function!')
        temp = self.feature_names + [self.target]
        pred = 0
        train, val, test = self.create_pred_train_test(period)
        train_ds = self.df_to_dataset(data = train)
        val_ds = self.df_to_dataset(data = val)

        self.train_and_evaluate_bnn(epochs, train_data = train_ds, val_data = val_ds)

        dataset_period_pred = self.df_to_dataset(data = test[temp], shuffle = False)
        feat, targets = list(dataset_period_pred.unbatch().batch(len(test)))[0]
        targets = targets.numpy()
        for i in range(0, iterations):
            pred += self.model(feat).numpy()
        rmse = math.sqrt(np.mean((pred*(1/iterations) - targets[0:, None])**2))
        mape = np.mean(np.abs(pred*(1/iterations) - targets[0:, None])/targets[0:, None])

        print('Prediction Error - RMSE: ' + str(rmse), 'MAPE: ' + str(mape))

        return pred*(1/iterations), test.index, rmse, mape

        
    def cal_estimation_errors(self, epochs, period = None, iterations = 50):
        """
            This function is doing about the same as the cal_prediction_error function but for the 
            estimation errors.
        """
        print('Model cannot be trained beforehand - delete trained model before calling this function!')
        temp = self.feature_names + [self.target]
        pred = 0
        train, val, test = self.create_est_train_test(period)
        train_ds = self.df_to_dataset(data = train)
        val_ds = self.df_to_dataset(data = val)

        self.train_and_evaluate_bnn(epochs, train_data = train_ds, val_data = val_ds)


        dataset_period_pred = self.df_to_dataset(data = test[temp], shuffle = False)
        feat, targets = list(dataset_period_pred.unbatch().batch(len(test)))[0]
        targets = targets.numpy()

        for i in range(0, iterations):
            pred += self.model(feat).numpy()


        rmse = math.sqrt(np.mean((pred*(1/iterations) - targets[0:, None])**2))
        mape = np.mean(np.abs(pred*(1/iterations) - targets[0:, None])/targets[0:, None])

        print('Prediction Error - RMSE: ' + str(rmse), 'MAPE: ' + str(mape))

        return pred*(1/iterations), test.index, rmse, mape

In [ ]:
class BNN():

    """
        This class defines the Bayesian Neural Network.
    """

    def __init__(self,
                 hidden_nodes,
                 learning_rate,
                 batch_size = 512,
                 alpha = 10**(-3),
                 loss = None):
      
        '''
            The init function defines the variables used in the Bayesian Neural Network.
        '''
        self.hidden_nodes = hidden_nodes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.alpha = alpha

        if loss == None:
          self.loss = keras.losses.MeanSquaredError()
        else:
          self.loss = loss
    
    def create_bnn_model(self, train_size, inputs):
        '''
            create_bnn_model defines the Bayesian Neural Network as described in our paper.
        '''
        features = keras.layers.concatenate(list(inputs.values()))
        features = layers.BatchNormalization()(features)

        layer_dict = {}
        initVec = [len(inputs)] + self.hidden_nodes
        i = 0
        
        for units in self.hidden_nodes:
            temp = layers.Dense(
                units=units,
                activation="relu",
                kernel_initializer=initializers.RandomNormal(stddev=np.sqrt(2)/np.sqrt(initVec[i])),
                kernel_regularizer = tf.keras.regularizers.l2(self.alpha)
            )
            layer_dict[i] = temp
            features = temp(features)
            features = layers.BatchNormalization()(features)
            i += 1
    
        outputs = layers.Dense(units=1, activation="linear",
                              kernel_regularizer = tf.keras.regularizers.l2(self.alpha))(features)
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        return model

#**Data preparation**

In [ ]:
option_data = pd.read_csv("/Users/svenspa/Downloads/data aqf/oex option data 2000-2020.csv")
underlying_data=pd.read_csv("/Users/svenspa/Downloads/data aqf/oex underlying.csv")
interest_rate_data=pd.read_csv("/Users/svenspa/Downloads/data aqf/yield curve.csv")
option_data_european = pd.read_csv("/Users/svenspa/Downloads/data aqf/xeo option data 2002-2020.csv")
underlying_data[['date']] = pd.to_datetime(underlying_data['date'],format='%Y%m%d')
underlying_data['date'] = underlying_data["date"].dt.date
interest_rate_data[['date']] = pd.to_datetime(interest_rate_data['date'],format='%Y%m%d')
interest_rate_data['date'] = interest_rate_data["date"].dt.date

In [ ]:
def process_data(option_data,underlying_data,interest_rate_data,is_American):
  #Remove options with the wrong date
  min_date=20030101
  max_date=20121231
  option_data=option_data[(option_data['date'] >= min_date) & (option_data['date'] <= max_date)]

  #Remove options below a certain volume
  min_volume=1
  option_data=option_data[option_data['volume'] >= min_volume]

  #Remove call options
  option_data=option_data[ option_data['cp_flag'] == "P" ]

  #Create a time to maturity column (in trading days)
  option_data[['date']] = pd.to_datetime(option_data['date'],format='%Y%m%d')
  option_data['date'] = option_data["date"].dt.date
  option_data[['exdate']] = pd.to_datetime(option_data['exdate'],format='%Y%m%d')
  option_data['exdate'] = option_data["exdate"].dt.date
  if(is_American):
    option_data[['last_date']] = pd.to_datetime(option_data['last_date'],format='%Y%m%d')
    option_data['last_date'] = option_data["last_date"].dt.date
  option_data['maturity']=np.busday_count(option_data['date'],option_data['exdate'], weekmask='1111111') #Do we have to include weekends?

  #Remove options that do not have correct maturity
  min_maturity=7
  max_maturity=90
  option_data = option_data[(option_data['maturity'] <= max_maturity) & (option_data['maturity'] >= min_maturity)]

  #Create a column with market prices
  option_data['market price'] = (option_data['best_bid'] + option_data['best_offer'])/2

  #Add the underlying to the dataframe
  rows=option_data.shape[0]
  underlying_vec=np.empty(rows, dtype=object)
  date_selector=0
  row_number=0
  for index, row in option_data.iterrows():
    while (underlying_data.at[date_selector,'date']!=row['date']):
      date_selector=date_selector+1
    underlying_vec[row_number]=underlying_data.at[date_selector,'close']
    row_number=row_number+1
  option_data['underlying']=underlying_vec

  #Create columns moneyness and time value
  option_data['moneyness'] = option_data['underlying'] / option_data['strike_price'] * 1000
  option_data['time value'] =  option_data['market price'] - (option_data['strike_price']/1000 - option_data['underlying'])

  #Create column with forward volume
  option_data['1d forward volume'] = option_data.groupby(['optionid'])['volume'].shift(-1)

  #Cleaning based on Ruf and Wang 2020 (https://github.com/weiguanwang/Hedging_Neural_Networks/tree/74d5a7b72626e92395d056ced01cb8e135edcbcc)
  option_data.dropna(subset=['impl_volatility'], how='any', inplace=True)
  option_data = option_data[(option_data['impl_volatility'] < 1) & (option_data['impl_volatility'] > 0.01)] 
  option_data = option_data[option_data['volume'] > 0]
  option_data = option_data[option_data['best_bid'] > 0.05]
  option_data = option_data[(option_data['time value']> 0)]
  option_data = option_data[ (2*option_data['best_bid'] > option_data['best_offer'])]
  option_data = option_data[~option_data['1d forward volume'].isna()]
  if(is_American):
    option_data = option_data[option_data['last_date'] == option_data['date']]

  #Add the interest to the dataframe -> maybe should scale it somehow
  rows=option_data.shape[0]
  interest_rate_vec=np.empty(rows, dtype=object)
  date_selector=0
  row_number=0
  for index, row in option_data.iterrows():
    while (interest_rate_data.at[date_selector,'date']<row['date']): #There are dates where options are traded, but no interest is quoted
      date_selector=date_selector+1
      if date_selector>=216968:
        print(date_selector)
        print(interest_rate_data.at[date_selector,'date'])
        print(row['date'])
    i=0
    while (interest_rate_data.at[date_selector+i,'days']<row['maturity']):
      i=i+1
    left_gap=row['maturity']-interest_rate_data.at[date_selector+i-1,'days']
    right_gap=interest_rate_data.at[date_selector+i,'days']-row['maturity']
    if left_gap<right_gap:
      i=i-1
    interest_rate_vec[row_number]=interest_rate_data.at[date_selector+i,'rate']
    row_number=row_number+1
  option_data['interest_rate']=interest_rate_vec

  option_data['maturity']=option_data['maturity']/250
  option_data['strike_price']=option_data['strike_price']/1000

  #Converting some of the columns from objects back to float values
  option_data = option_data.astype({'moneyness': 'float64'})

  option_data = option_data.reset_index(drop=True)

  return option_data

In [ ]:
option_data=process_data(option_data,underlying_data,interest_rate_data,True)
option_data_european=process_data(option_data_european,underlying_data,interest_rate_data,False)

**See the performance of the Bayesian Neural Network**

In [ ]:
pricevecEst = np.empty((103496,1))
pricevecEst[:] = np.NaN
for per in ['pre-crisis','crisis','post-crisis']:
    NN = ANNOptionPricing(option_data, 
                              ['moneyness','maturity'],
                              'market price',
                              hidden_nodes = [64, 256, 128],
                              learning_rate = 0.01,
                              batch_size = 256,
                              alpha=0.000001,
                              beta = 1)
    predvec, index, rmse, mape = NN.cal_estimation_errors(period=per, epochs = 500)
    del NN.model
    pricevecEst[index] = predvec

Model cannot be trained beforehand - delete trained model before calling this function!
Start training the model...
Epoch 1/500
122/122 - 2s - loss: 4114.7188 - root_mean_squared_error: 4.0118 - val_loss: 11506.2188 - val_root_mean_squared_error: 6.7547
Epoch 2/500
122/122 - 0s - loss: 2204.6816 - root_mean_squared_error: 2.9369 - val_loss: 8833.3809 - val_root_mean_squared_error: 5.9015
Epoch 3/500
122/122 - 0s - loss: 1998.0488 - root_mean_squared_error: 2.7967 - val_loss: 5873.4932 - val_root_mean_squared_error: 4.8433
Epoch 4/500
122/122 - 0s - loss: 1944.7903 - root_mean_squared_error: 2.7581 - val_loss: 2273.2063 - val_root_mean_squared_error: 3.0030
Epoch 5/500
122/122 - 0s - loss: 1841.6603 - root_mean_squared_error: 2.6858 - val_loss: 2242.4929 - val_root_mean_squared_error: 2.9869
Epoch 6/500
122/122 - 0s - loss: 1788.0050 - root_mean_squared_error: 2.6465 - val_loss: 1893.0212 - val_root_mean_squared_error: 2.7419
Epoch 7/500
122/122 - 0s - loss: 1795.3203 - root_mean_square

Epoch 60/500
122/122 - 0s - loss: 1572.2426 - root_mean_squared_error: 2.4796 - val_loss: 559.3679 - val_root_mean_squared_error: 1.4882
Epoch 61/500
122/122 - 0s - loss: 1569.2087 - root_mean_squared_error: 2.4779 - val_loss: 1023.2559 - val_root_mean_squared_error: 2.0175
Epoch 62/500
122/122 - 0s - loss: 1527.3436 - root_mean_squared_error: 2.4447 - val_loss: 829.8324 - val_root_mean_squared_error: 1.8209
Epoch 63/500
122/122 - 0s - loss: 1565.7125 - root_mean_squared_error: 2.4755 - val_loss: 490.9179 - val_root_mean_squared_error: 1.3949
Epoch 64/500
122/122 - 0s - loss: 1519.0118 - root_mean_squared_error: 2.4387 - val_loss: 981.3754 - val_root_mean_squared_error: 1.9739
Epoch 65/500
122/122 - 0s - loss: 1492.3318 - root_mean_squared_error: 2.4172 - val_loss: 1120.6896 - val_root_mean_squared_error: 2.1137
Epoch 66/500
122/122 - 0s - loss: 1630.4044 - root_mean_squared_error: 2.5260 - val_loss: 903.9549 - val_root_mean_squared_error: 1.8916
Epoch 67/500
122/122 - 0s - loss: 1557.

Epoch 120/500
122/122 - 0s - loss: 1499.5706 - root_mean_squared_error: 2.4226 - val_loss: 2063.4727 - val_root_mean_squared_error: 2.8618
Epoch 121/500
122/122 - 0s - loss: 1531.4324 - root_mean_squared_error: 2.4484 - val_loss: 1951.7202 - val_root_mean_squared_error: 2.7812
Epoch 122/500
122/122 - 0s - loss: 1564.8926 - root_mean_squared_error: 2.4741 - val_loss: 391.0461 - val_root_mean_squared_error: 1.2542
Epoch 123/500
122/122 - 0s - loss: 1444.9408 - root_mean_squared_error: 2.3785 - val_loss: 587.4048 - val_root_mean_squared_error: 1.5345
Epoch 124/500
122/122 - 0s - loss: 1562.9667 - root_mean_squared_error: 2.4723 - val_loss: 672.9687 - val_root_mean_squared_error: 1.6363
Epoch 125/500
122/122 - 0s - loss: 1479.2360 - root_mean_squared_error: 2.4062 - val_loss: 664.2927 - val_root_mean_squared_error: 1.6211
Epoch 126/500
122/122 - 0s - loss: 1454.6819 - root_mean_squared_error: 2.3855 - val_loss: 860.1398 - val_root_mean_squared_error: 1.8456
Epoch 127/500
122/122 - 0s - los

Epoch 180/500
122/122 - 0s - loss: 1517.6589 - root_mean_squared_error: 2.4393 - val_loss: 1050.7650 - val_root_mean_squared_error: 2.0422
Epoch 181/500
122/122 - 0s - loss: 1502.7222 - root_mean_squared_error: 2.4249 - val_loss: 1467.4192 - val_root_mean_squared_error: 2.4129
Epoch 182/500
122/122 - 0s - loss: 1520.2491 - root_mean_squared_error: 2.4385 - val_loss: 1167.2836 - val_root_mean_squared_error: 2.1502
Epoch 183/500
122/122 - 0s - loss: 1509.9871 - root_mean_squared_error: 2.4308 - val_loss: 719.9778 - val_root_mean_squared_error: 1.6912
Epoch 184/500
122/122 - 0s - loss: 1498.8319 - root_mean_squared_error: 2.4214 - val_loss: 1437.1288 - val_root_mean_squared_error: 2.3903
Epoch 185/500
122/122 - 0s - loss: 1514.2800 - root_mean_squared_error: 2.4360 - val_loss: 842.5869 - val_root_mean_squared_error: 1.8306
Epoch 186/500
122/122 - 0s - loss: 1485.8571 - root_mean_squared_error: 2.4136 - val_loss: 1495.0663 - val_root_mean_squared_error: 2.4337
Epoch 187/500
122/122 - 0s - 

Epoch 240/500
122/122 - 0s - loss: 1537.3986 - root_mean_squared_error: 2.4535 - val_loss: 823.0854 - val_root_mean_squared_error: 1.8047
Epoch 241/500
122/122 - 0s - loss: 1501.0132 - root_mean_squared_error: 2.4233 - val_loss: 1086.7629 - val_root_mean_squared_error: 2.0774
Epoch 242/500
122/122 - 0s - loss: 1504.6605 - root_mean_squared_error: 2.4276 - val_loss: 932.2160 - val_root_mean_squared_error: 1.9226
Epoch 243/500
122/122 - 0s - loss: 1473.8624 - root_mean_squared_error: 2.4013 - val_loss: 913.4991 - val_root_mean_squared_error: 1.9055
Epoch 244/500
122/122 - 0s - loss: 1514.2756 - root_mean_squared_error: 2.4370 - val_loss: 1090.6653 - val_root_mean_squared_error: 2.0821
Epoch 245/500
122/122 - 0s - loss: 1521.1039 - root_mean_squared_error: 2.4409 - val_loss: 939.7392 - val_root_mean_squared_error: 1.9309
Epoch 246/500
122/122 - 0s - loss: 1494.1409 - root_mean_squared_error: 2.4170 - val_loss: 535.2619 - val_root_mean_squared_error: 1.4565
Epoch 247/500
122/122 - 0s - los

...

Epoch 250/500
389/389 - 1s - loss: 5426.6299 - root_mean_squared_error: 4.6056 - val_loss: 1368.8080 - val_root_mean_squared_error: 2.3304
Epoch 251/500
389/389 - 1s - loss: 5417.4478 - root_mean_squared_error: 4.6013 - val_loss: 618.4853 - val_root_mean_squared_error: 1.5656
Epoch 252/500
389/389 - 1s - loss: 5597.1128 - root_mean_squared_error: 4.6772 - val_loss: 1741.3389 - val_root_mean_squared_error: 2.6282
Epoch 253/500
389/389 - 1s - loss: 5585.5762 - root_mean_squared_error: 4.6721 - val_loss: 829.9211 - val_root_mean_squared_error: 1.8111
Epoch 254/500
389/389 - 1s - loss: 5450.6196 - root_mean_squared_error: 4.6153 - val_loss: 597.7558 - val_root_mean_squared_error: 1.5354
Epoch 255/500
389/389 - 1s - loss: 5539.8555 - root_mean_squared_error: 4.6528 - val_loss: 819.1997 - val_root_mean_squared_error: 1.7991
Epoch 256/500
389/389 - 1s - loss: 5425.0615 - root_mean_squared_error: 4.6046 - val_loss: 635.8947 - val_root_mean_squared_error: 1.5817
Epoch 257/500
389/389 - 1s - los

Epoch 310/500
389/389 - 1s - loss: 5443.9912 - root_mean_squared_error: 4.6125 - val_loss: 663.6169 - val_root_mean_squared_error: 1.6162
Epoch 311/500
389/389 - 1s - loss: 5606.9507 - root_mean_squared_error: 4.6811 - val_loss: 716.7219 - val_root_mean_squared_error: 1.6880
Epoch 312/500
389/389 - 1s - loss: 5458.3745 - root_mean_squared_error: 4.6193 - val_loss: 742.8013 - val_root_mean_squared_error: 1.7100
Epoch 313/500
389/389 - 1s - loss: 5494.7690 - root_mean_squared_error: 4.6356 - val_loss: 911.9749 - val_root_mean_squared_error: 1.8966
Epoch 314/500
389/389 - 1s - loss: 5422.0688 - root_mean_squared_error: 4.6037 - val_loss: 1130.7476 - val_root_mean_squared_error: 2.1128
Epoch 315/500
389/389 - 1s - loss: 5483.0698 - root_mean_squared_error: 4.6300 - val_loss: 1057.5391 - val_root_mean_squared_error: 2.0444
Epoch 316/500
389/389 - 1s - loss: 5552.3960 - root_mean_squared_error: 4.6578 - val_loss: 1032.2896 - val_root_mean_squared_error: 2.0192
Epoch 317/500
389/389 - 1s - lo

Epoch 370/500
389/389 - 1s - loss: 5397.4873 - root_mean_squared_error: 4.5933 - val_loss: 1002.7806 - val_root_mean_squared_error: 1.9854
Epoch 371/500
389/389 - 1s - loss: 5436.0835 - root_mean_squared_error: 4.6100 - val_loss: 1199.8175 - val_root_mean_squared_error: 2.1733
Epoch 372/500
389/389 - 1s - loss: 5454.0952 - root_mean_squared_error: 4.6171 - val_loss: 729.4098 - val_root_mean_squared_error: 1.7198
Epoch 373/500
389/389 - 1s - loss: 5513.4961 - root_mean_squared_error: 4.6424 - val_loss: 1465.8751 - val_root_mean_squared_error: 2.4103
Epoch 374/500
389/389 - 1s - loss: 5297.4829 - root_mean_squared_error: 4.5505 - val_loss: 558.7117 - val_root_mean_squared_error: 1.4857
Epoch 375/500
389/389 - 1s - loss: 5387.1230 - root_mean_squared_error: 4.5888 - val_loss: 1060.3621 - val_root_mean_squared_error: 2.0486
Epoch 376/500
389/389 - 1s - loss: 5557.6475 - root_mean_squared_error: 4.6605 - val_loss: 606.6140 - val_root_mean_squared_error: 1.5521
Epoch 377/500
389/389 - 1s - l

Epoch 430/500
389/389 - 1s - loss: 5473.6934 - root_mean_squared_error: 4.6249 - val_loss: 1038.1716 - val_root_mean_squared_error: 2.0237
Epoch 431/500
389/389 - 1s - loss: 5473.5752 - root_mean_squared_error: 4.6262 - val_loss: 798.9969 - val_root_mean_squared_error: 1.7847
Epoch 432/500
389/389 - 1s - loss: 5454.2759 - root_mean_squared_error: 4.6165 - val_loss: 761.0782 - val_root_mean_squared_error: 1.7304
Epoch 433/500
389/389 - 1s - loss: 5582.7944 - root_mean_squared_error: 4.6720 - val_loss: 765.8799 - val_root_mean_squared_error: 1.7390
Epoch 434/500
389/389 - 1s - loss: 5407.7710 - root_mean_squared_error: 4.5971 - val_loss: 889.5482 - val_root_mean_squared_error: 1.8894
Epoch 435/500
389/389 - 1s - loss: 5420.7988 - root_mean_squared_error: 4.6047 - val_loss: 1827.4709 - val_root_mean_squared_error: 2.6871
Epoch 436/500
389/389 - 1s - loss: 5296.2119 - root_mean_squared_error: 4.5498 - val_loss: 759.2108 - val_root_mean_squared_error: 1.7306
Epoch 437/500
389/389 - 1s - los

Epoch 490/500
389/389 - 1s - loss: 5342.1284 - root_mean_squared_error: 4.5698 - val_loss: 899.8326 - val_root_mean_squared_error: 1.8820
Epoch 491/500
389/389 - 1s - loss: 5410.9980 - root_mean_squared_error: 4.5996 - val_loss: 569.8627 - val_root_mean_squared_error: 1.5048
Epoch 492/500
389/389 - 1s - loss: 5411.0635 - root_mean_squared_error: 4.6007 - val_loss: 983.0853 - val_root_mean_squared_error: 1.9741
Epoch 493/500
389/389 - 1s - loss: 5465.0254 - root_mean_squared_error: 4.6241 - val_loss: 706.7565 - val_root_mean_squared_error: 1.6701
Epoch 494/500
389/389 - 1s - loss: 5360.1982 - root_mean_squared_error: 4.5791 - val_loss: 1017.4195 - val_root_mean_squared_error: 2.0068
Epoch 495/500
389/389 - 1s - loss: 5490.6606 - root_mean_squared_error: 4.6321 - val_loss: 1099.9204 - val_root_mean_squared_error: 2.0845
Epoch 496/500
389/389 - 1s - loss: 5325.0654 - root_mean_squared_error: 4.5621 - val_loss: 1553.8641 - val_root_mean_squared_error: 2.4757
Epoch 497/500
389/389 - 1s - lo